## Load Libs

In [ ]:
import os
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/open_directory/test/IMAGE/dataset')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/open_directory/test/IMAGE/dataset


In [ ]:
ls

large_unet_fold0_0.9024.pth  unet_fold0_0.9108.pth
model.h5                     unet_fold0_new_metric_0.8925.pth
submission.csv               uw3dweights.zip
unet_fold0_0.8789.pth        uw-madison-gi-tract-image-segmentation@
unet_fold0_0.8932.pth


In [ ]:
pip install 'monai[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 939 kB 31.4 MB/s 
     |████████████████████████████████| 17.8 MB 81.5 MB/s 
     |████████████████████████████████| 8.4 MB 37.1 MB/s 
     |████████████████████████████████| 251 kB 8.6 MB/s 
     |████████████████████████████████| 4.2 MB 61.6 MB/s 
     |████████████████████████████████| 31.0 MB 1.3 MB/s 
     |████████████████████████████████| 316 kB 76.3 MB/s 
     |████████████████████████████████| 125 kB 84.6 MB/s 
     |████████████████████████████████| 87 kB 8.3 MB/s 
     |████████████████████████████████| 15.0 MB 36.1 MB/s 
     |████████████████████████████████| 70.6 MB 16 kB/s 
     |████████████████████████████████| 16.6 MB 33.4 MB/s 
     |████████████████████████████████| 20.3 MB 145 kB/s 
     |████████████████████████████████| 95.3 MB 54 kB/s 
     |████████████████████████████████| 54.5 MB 1.2 MB/s 
     |███████████████████████████

In [ ]:
!pip install kaggle --upgrade
os.environ['KAGGLE_USERNAME'] = 'daldarong'
os.environ['KAGGLE_KEY'] = '34a4da55aa9665a8317bdc6de625793d'
!kaggle kernels output yiheng/3d-solution-with-monai-infer -p /path/to/dest

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Output file downloaded to /path/to/dest/submission.csv
Kernel log downloaded to /path/to/dest/3d-solution-with-monai-infer.log 


In [ ]:
import gc
from glob import glob
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
from monai.inferers import sliding_window_inference
from monai.data import decollate_batch
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
import json
import pandas as pd

In [ ]:
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    Compose,
    Activations,
    AsDiscrete,
    Activationsd,
    AsDiscreted,
    KeepLargestConnectedComponentd,
    Invertd,
    LoadImage,
    Transposed,
    LoadImaged,
    AddChanneld,
    CastToTyped,
    Lambdad,
    Resized,
    EnsureTyped,
    SpatialPadd,
    EnsureChannelFirstd,
)

In [ ]:
ls

large_unet_fold0_0.9024.pth  unet_fold0_0.9108.pth
model.h5                     unet_fold0_new_metric_0.8925.pth
submission.csv               uw3dweights.zip
unet_fold0_0.8789.pth        uw-madison-gi-tract-image-segmentation@
unet_fold0_0.8932.pth


In [ ]:
pd.read_csv('./uw-madison-gi-tract-image-segmentation/train.csv')

,id,class,segmentation
0,case123_day20_slice_0001,large_bowel,NaN
1,case123_day20_slice_0001,small_bowel,NaN
2,case123_day20_slice_0001,stomach,NaN
3,case123_day20_slice_0002,large_bowel,NaN
4,case123_day20_slice_0002,small_bowel,NaN
...,...,...,...
115483,case30_day0_slice_0143,small_bowel,NaN
115484,case30_day0_slice_0143,stomach,NaN
115485,case30_day0_slice_0144,large_bowel,NaN
115486,case30_day0_slice_0144,small_bowel,NaN


## Prepare meta info.

### Thanks awsaf49, this section refers to:
https://www.kaggle.com/code/awsaf49/uwmgi-2-5d-infer-pytorch

In [ ]:
def get_metadata(row):
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

def path2info(row):
    path = row['image_path']
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

In [ ]:
sub_df = pd.read_csv('uw-madison-gi-tract-image-segmentation/sample_submission.csv')
if not len(sub_df):
    debug = True
    sub_df = pd.read_csv('uw-madison-gi-tract-image-segmentation/train.csv')[:1000*3]
    sub_df = sub_df.drop(columns=['class','segmentation']).drop_duplicates()
else:
    debug = False
    sub_df = sub_df.drop(columns=['class','predicted']).drop_duplicates()
sub_df = sub_df.apply(lambda x: get_metadata(x),axis=1)

In [ ]:
if debug:
    paths = glob(f'uw-madison-gi-tract-image-segmentation/train/**/*png',recursive=True)
#     paths = sorted(paths)
else:
    paths = glob(f'uw-madison-gi-tract-image-segmentation/test/**/*png',recursive=True)
#     paths = sorted(paths)
path_df = pd.DataFrame(paths, columns=['image_path'])
path_df = path_df.apply(lambda x: path2info(x),axis=1)
path_df.head()

## Produce 3d data list for MONAI DataSet

In [ ]:
test_df = sub_df.merge(path_df, on=['case','day','slice'], how='left')
test_df["case_id_str"] = test_df["id"].apply(lambda x: x.split("_", 2)[0])
test_df["day_num_str"] = test_df["id"].apply(lambda x: x.split("_", 2)[1])
test_df["slice_id"] = test_df["id"].apply(lambda x: x.split("_", 2)[2])

In [ ]:
test_df.head()

,id,case,day,slice,image_path,height,width,case_id_str,day_num_str,slice_id
0,case123_day20_slice_0001,123,20,1,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case123,day20,slice_0001
1,case123_day20_slice_0002,123,20,2,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case123,day20,slice_0002
2,case123_day20_slice_0003,123,20,3,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case123,day20,slice_0003
3,case123_day20_slice_0004,123,20,4,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case123,day20,slice_0004
4,case123_day20_slice_0005,123,20,5,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case123,day20,slice_0005


In [ ]:
test_data = []

for group in test_df.groupby(["case_id_str", "day_num_str"]):

    case_id_str, day_num_str = group[0]
    group_id = case_id_str + "_" + day_num_str
    group_df = group[1].sort_values("slice_id", ascending=True)
    n_slices = group_df.shape[0]
    group_slices, group_ids = [], []
    for idx in range(n_slices):
        slc = group_df.iloc[idx]
        group_slices.append(slc.image_path)
        group_ids.append(slc.id)
    test_data.append({"image": group_slices, "id": group_ids})

In [ ]:
len(test_data)

7

In [ ]:
group[0], group[1]

('case77', 'day20')
                          id  case  day  slice  \
432  case77_day20_slice_0001    77   20      1   
433  case77_day20_slice_0002    77   20      2   
434  case77_day20_slice_0003    77   20      3   
435  case77_day20_slice_0004    77   20      4   
436  case77_day20_slice_0005    77   20      5   
..                       ...   ...  ...    ...   
571  case77_day20_slice_0140    77   20    140   
572  case77_day20_slice_0141    77   20    141   
573  case77_day20_slice_0142    77   20    142   
574  case77_day20_slice_0143    77   20    143   
575  case77_day20_slice_0144    77   20    144   

                                            image_path  height  width  \
432  uw-madison-gi-tract-image-segmentation/train/c...     266    266   
433  uw-madison-gi-tract-image-segmentation/train/c...     266    266   
434  uw-madison-gi-tract-image-segmentation/train/c...     266    266   
435  uw-madison-gi-tract-image-segmentation/train/c...     266    266   
436  uw-madiso

In [ ]:
group_df

,id,case,day,slice,image_path,height,width,case_id_str,day_num_str,slice_id
432,case77_day20_slice_0001,77,20,1,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case77,day20,slice_0001
433,case77_day20_slice_0002,77,20,2,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case77,day20,slice_0002
434,case77_day20_slice_0003,77,20,3,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case77,day20,slice_0003
435,case77_day20_slice_0004,77,20,4,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case77,day20,slice_0004
436,case77_day20_slice_0005,77,20,5,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case77,day20,slice_0005
...,...,...,...,...,...,...,...,...,...,...
571,case77_day20_slice_0140,77,20,140,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case77,day20,slice_0140
572,case77_day20_slice_0141,77,20,141,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case77,day20,slice_0141
573,case77_day20_slice_0142,77,20,142,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case77,day20,slice_0142
574,case77_day20_slice_0143,77,20,143,uw-madison-gi-tract-image-segmentation/train/c...,266,266,case77,day20,slice_0143


In [ ]:
n_slices

144

## Prepare Transforms, Dataset, DataLoader

In [ ]:
class cfg:
    img_size = (224, 224, 80)
    in_channels = 1
    out_channels = 3
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    weights = glob("large*")
    batch_size = 1
    sw_batch_size = 4
    print(device)

cuda:0


In [ ]:
cfg.weights

['large_unet_fold0_0.9024.pth']

In [ ]:
ls

model.h5  uw-madison-gi-tract-image-segmentation@


In [ ]:
cfg.weights

[]

In [ ]:
test_transforms = Compose(
    [
        LoadImaged(keys="image"), # d, h, w
        AddChanneld(keys="image"), # c, d, h, w
        Transposed(keys="image", indices=[0, 2, 3, 1]), # c, w, h, d
        Lambdad(keys="image", func=lambda x: x / x.max()),
#         SpatialPadd(keys="image", spatial_size=cfg.img_size),  # in case less than 80 slices
        EnsureTyped(keys="image", dtype=torch.float32),
    ]
)

test_ds = CacheDataset(
        data=test_data,
        transform=test_transforms,
        cache_rate=0.0,
        num_workers=2,
    )

test_dataloader = DataLoader(
    test_ds,
    batch_size=cfg.batch_size,
    num_workers=2,
    pin_memory=True,
)

In [ ]:
test_transforms

In [ ]:
test_ds

In [ ]:
test_dataloader

## Prepare Network

In [ ]:
model = UNet(
    spatial_dims=3,
    in_channels=cfg.in_channels,
    out_channels=cfg.out_channels,
    channels=(32, 64, 128, 256, 512),
    strides=(2, 2, 2, 2),
    kernel_size=3,
    up_kernel_size=3,
    num_res_units=2,
    act="PRELU",
    norm="BATCH",
    dropout=0.2,
    bias=True,
    dimensions=None,
).to(cfg.device)

## Infer

In [ ]:
# ref.: https://www.kaggle.com/stainsby/fast-tested-rle
def rle_encode(img):
    """ TBD
    
    Args:
        img (np.array): 
            - 1 indicating mask
            - 0 indicating background
    
    Returns: 
        run length as string formated
    """
    
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
if list_of_tensors:
    t = torch.stack(list_of_tensors)
else
    t = torch.tensor([])

In [ ]:
!kaggle datasets download -d yiheng/uw3dweights

100% 136M/136M [00:04<00:00, 40.3MB/s]
100% 136M/136M [00:04<00:00, 29.6MB/s]


In [ ]:
!unzip 'uw3dweights.zip'

Archive:  uw3dweights.zip
  inflating: large_unet_fold0_0.9024.pth  
  inflating: unet_fold0_0.8789.pth   
  inflating: unet_fold0_0.8932.pth   
  inflating: unet_fold0_0.9108.pth   
  inflating: unet_fold0_new_metric_0.8925.pth  


In [ ]:
ls

large_unet_fold0_0.9024.pth  unet_fold0_0.9108.pth
model.h5                     unet_fold0_new_metric_0.8925.pth
unet_fold0_0.8789.pth        uw3dweights.zip
unet_fold0_0.8932.pth        uw-madison-gi-tract-image-segmentation@


In [ ]:
outputs = []

post_pred = Compose([
    Activations(sigmoid=True),
    AsDiscrete(threshold=0.5),
])

model.eval()
torch.set_grad_enabled(False)
progress_bar = tqdm(range(len(test_dataloader)))
val_it = iter(test_dataloader)
for itr in progress_bar:
    batch = next(val_it)
    test_inputs = batch["image"].to(cfg.device)

    pred_all = []
    for weights in cfg.weights:
        model.load_state_dict(torch.load(weights))
        pred = sliding_window_inference(test_inputs, cfg.img_size, cfg.sw_batch_size, model)
        pred_all.append(pred)
        # do 4 tta
        for dims in [[2], [3], [2, 3]]:
            flip_pred = sliding_window_inference(torch.flip(test_inputs, dims=dims), cfg.img_size, cfg.sw_batch_size, model)
            flip_pred = torch.flip(flip_pred, dims=dims)
            pred_all.append(flip_pred)
    
    pred_all = torch.mean(torch.stack(pred_all), dim=0)[0]
    pred_all = post_pred(pred_all)
    # c, w, h, d to d, c, h, w
    pred_all = torch.permute(pred_all, [3, 0, 2, 1]).cpu().numpy().astype(np.uint8)
    id_outputs = from_engine(["id"])(batch)[0]

    for test_output, id_output in zip(pred_all, id_outputs):
        id_name = id_output[0]
        lb, sb, st = test_output
        outputs.append([id_name, "large_bowel", rle_encode(lb)])
        outputs.append([id_name, "small_bowel", rle_encode(sb)])
        outputs.append([id_name, "stomach", rle_encode(st)])


100%|██████████| 7/7 [01:47<00:00, 15.33s/it]


In [ ]:
cfg.weights

['large_unet_fold0_0.9024.pth']

In [ ]:
post_pred

In [ ]:
submit = pd.DataFrame(data=np.array(outputs), columns=["id", "class", "predicted"])

In [ ]:
# Fix sub error, refers to: https://www.kaggle.com/competitions/uw-madison-gi-tract-image-segmentation/discussion/320541
if not debug:
    sub_df = pd.read_csv('uw-madison-gi-tract-image-segmentation/sample_submission.csv')
    del sub_df['predicted']
    sub_df = sub_df.merge(submit, on=['id','class'])
    sub_df.to_csv('submission.csv',index=False)
else:
    submit.to_csv('submission.csv', index=False)

In [ ]:
df1 = pd.read_csv('submission.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         3000 non-null   object
 1   class      3000 non-null   object
 2   predicted  819 non-null    object
dtypes: object(3)
memory usage: 70.4+ KB
